In [1]:
import pandas as pd
from dont_patronize_me import DontPatronizeMe
from ast import literal_eval
from torch.utils.data import DataLoader, Dataset
import os.path as osp
from typing import Optional
from transformer import RoBERTa
from tqdm import tqdm
import pandas as pd
import torch
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader, Dataset
from transformers.models.auto.tokenization_auto import AutoTokenizer
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = 'cuda'

In [2]:
class DPMDataset_submission(Dataset):
    def __init__(self, model: str, data) -> None:
        super().__init__()

        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.max_len = 512
        self.data = data

    def __getitem__(self, index):
        text = self.data[index]
        text = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True,
        )
        return {
            "ids": torch.tensor(text["input_ids"], dtype=torch.long),
            "mask": torch.tensor(text["attention_mask"], dtype=torch.long),
        }

    def __len__(self):
        return len(self.data)

In [3]:
dpm = DontPatronizeMe('../dataset', test_path="../dataset/task4_test.tsv")
dpm.load_test()

In [4]:
dpm.load_test()
test = dpm.test_set

In [5]:
len(test)

3832

In [6]:
dataset_ = DPMDataset_submission(model = "mrm8488/distilroberta-finetuned-tweets-hate-speech",data=dpm.test_set)

In [7]:
data_loader = DataLoader(dataset_, batch_size=8)

In [8]:
best_model_path = "models/distillroberta_3/bert-val_loss0.44-v1.ckpt"
hf_name = "mrm8488/distilroberta-finetuned-tweets-hate-speech"
module = RoBERTa(model=hf_name).load_from_checkpoint(best_model_path, model=hf_name)


mrm8488/distilroberta-finetuned-tweets-hate-speech
2048
mrm8488/distilroberta-finetuned-tweets-hate-speech
2048


In [9]:
module.eval()
module.freeze()
module.cuda()
preds = []
sub_file = "text.txt"
file_path = "../submissions/task2.txt"
f = open(file_path, "w")
for batch in tqdm(data_loader):
    ids,mask = batch["ids"].cuda(), batch["mask"].cuda()
    out = module(ids,mask)
    # print(out)
    predictions = torch.sigmoid(out).cpu()
    predictions = torch.round(predictions)
    # print(predictions)
    predictions = predictions.numpy().astype(int).tolist()
    # print(predictions)
    for i, pred in enumerate(predictions):
        result  = ",".join([str(x) for x in pred]).strip('"') + "\n"
        f.write(result)
        # print(result)
        preds.append(result)
    # break
f.close()

100%|██████████| 479/479 [00:25<00:00, 18.55it/s]
